In [2]:
import pandas as pd
from pathlib import Path
import wfdb

# Read lab-event
lab = pd.read_excel(
    'labevents-1000.xlsx',
    usecols=['subject_id', 'charttime', 'itemid', 'value'],
    parse_dates=['charttime']
)
# Keep Ca²⁺(50893) and K⁺(50971)
lab = lab[lab['itemid'].isin([50893, 50971])].copy()
lab.sort_values(['subject_id', 'charttime'], inplace=True)


In [3]:
# Check whether both Ca and K are present at each time point
# Count unique itemid and total count at each (subject_id, charttime)
lab_grouped = lab.groupby(['subject_id', 'charttime'])['itemid'].agg(['nunique', 'count'])
print("Distribution of test types over time points(1 both, 2 only one):")
print(lab_grouped['nunique'].value_counts())


Distribution of test types over time points(1 both, 2 only one):
nunique
2    19257
1     8639
Name: count, dtype: int64


In [ ]:
print(lab)

         subject_id  itemid           charttime value
18         10000032   50893 2180-03-23 11:51:00   8.4
39         10000032   50971 2180-03-23 11:51:00   3.7
107        10000032   50971 2180-05-06 22:25:00   3.4
147        10000032   50893 2180-05-07 05:05:00   7.8
153        10000032   50971 2180-05-07 05:05:00   4.5
...             ...     ...                 ...   ...
1048469    10070928   50971 2187-01-12 06:30:00   4.3
1048482    10070928   50971 2187-01-31 11:40:00   4.5
1048529    10070928   50893 2187-02-01 04:07:00   7.7
1048535    10070928   50971 2187-02-01 04:07:00   3.5
1048570    10070928   50893 2187-02-02 06:45:00   8.2

[47183 rows x 4 columns]


In [7]:
# Extract ECG features
records = []
root = Path('C:\\Users\\DELL\\Desktop\\DataECG')

In [9]:
for hea_path in root.rglob('*.hea'):
    if not hea_path.exists():
        continue

    parts = hea_path.parts
    try:
        subject_id = int(parts[-3][1:])  # 'p10000032' - 10000032
    except ValueError:
        continue
    study_id = parts[-2][1:]           # 's40689238' - '40689238'

    # Only keep subjects present in lab-event
    if subject_id not in set(lab['subject_id']):
        continue

    record_base = str(hea_path.with_suffix(''))  # 去掉 “.hea”

    try:
        hdr = wfdb.rdheader(record_base)
        ecg_time = pd.to_datetime(f"{hdr.base_date} {hdr.base_time}")
    except Exception as e:
        print(f"Fail to read ECG header, skip {hea_path.name}:{e}")
        continue

    records.append({
        'subject_id': subject_id,
        'study_id'  : study_id,
        'ecg_time'  : ecg_time,
        'hea_path'  : str(hea_path)
    })

ecg_df = pd.DataFrame(records)


In [10]:
print(ecg_df)

      subject_id  study_id            ecg_time  \
0       10000032  40689238 2180-07-23 08:44:00   
1       10000032  44458630 2180-07-23 09:54:00   
2       10000032  49036311 2180-08-06 09:07:00   
3       10000117  45090959 2181-03-04 17:14:00   
4       10000117  48446569 2183-09-18 13:52:00   
...          ...       ...                 ...   
7440    10070928  43278504 2187-12-05 13:34:00   
7441    10070928  44396383 2187-12-05 13:38:00   
7442    10070928  44703023 2187-01-31 11:56:00   
7443    10070928  45989652 2187-06-07 11:17:00   
7444    10070928  47084135 2187-08-24 19:16:00   

                                               hea_path  
0     C:\Users\DELL\Desktop\DataECG\p1000\p10000032\...  
1     C:\Users\DELL\Desktop\DataECG\p1000\p10000032\...  
2     C:\Users\DELL\Desktop\DataECG\p1000\p10000032\...  
3     C:\Users\DELL\Desktop\DataECG\p1000\p10000117\...  
4     C:\Users\DELL\Desktop\DataECG\p1000\p10000117\...  
...                                                

In [11]:
# subject_id intersection
common_ids = set(ecg_df['subject_id']) & set(lab['subject_id'])
ecg_df = ecg_df[ecg_df['subject_id'].isin(common_ids)].copy()
lab    = lab[   lab['subject_id'].isin(common_ids)].copy()

ecg_df.sort_values(['subject_id', 'ecg_time'], inplace=True)
lab.sort_values(['subject_id', 'charttime'], inplace=True)


In [12]:
print(lab.head)

<bound method NDFrame.head of          subject_id  itemid           charttime value
18         10000032   50893 2180-03-23 11:51:00   8.4
39         10000032   50971 2180-03-23 11:51:00   3.7
107        10000032   50971 2180-05-06 22:25:00   3.4
147        10000032   50893 2180-05-07 05:05:00   7.8
153        10000032   50971 2180-05-07 05:05:00   4.5
...             ...     ...                 ...   ...
1048469    10070928   50971 2187-01-12 06:30:00   4.3
1048482    10070928   50971 2187-01-31 11:40:00   4.5
1048529    10070928   50893 2187-02-01 04:07:00   7.7
1048535    10070928   50971 2187-02-01 04:07:00   3.5
1048570    10070928   50893 2187-02-02 06:45:00   8.2

[34660 rows x 4 columns]>


In [ ]:
# Turn into datetime
lab['charttime'] = pd.to_datetime(lab['charttime'])
ecg_df['ecg_time'] = pd.to_datetime(ecg_df['ecg_time'])

# Sort(necessary)
lab_sorted = lab.sort_values('charttime')
ecg_sorted = ecg_df.sort_values('ecg_time')

# Match the nearest ecg_time
merged = pd.merge_asof(
    left=lab_sorted,                                
    right=ecg_sorted,                               
    left_on='charttime',                           
    right_on='ecg_time',                           
    by='subject_id',                               
    direction='nearest',                           
    suffixes=('_lab', '_ecg')                      
)

merged['time_diff'] = abs(merged['charttime'] - merged['ecg_time'])

# Max time difference, set 24 here.
# Note: Ideally within 6 hours for greater accuracy, since blood test results in ICU patients may fluctuate within a single day, 
# with the most frequent testing occurring every 6 hours.
max_time_diff = pd.Timedelta(hours=24)
merged = merged[merged['time_diff'] <= max_time_diff].copy()

print("\nStatistics Summary:")
print(f"- Total Lab records: {len(lab)}")
print(f"- Matched Lab records: {len(merged)}")
print(f"- Matched Rate: {len(merged)/len(lab):.1%}")
print(f"- Max time difference: {merged['time_diff'].max()}")


Statistics Summary:
- Total Lab records: 34660
- Matched Lab records: 9604
- Matched Rate: 27.7%
- Max time difference: 1 days 00:00:00


In [14]:
# After first round cleaning, each ECG matched multiple blood tests.
# Requires a second round of cleaning.
# Seperate Ca and K records
ca_df = merged[merged['itemid'] == 50893].copy()
k_df = merged[merged['itemid'] == 50971].copy()

# Sort by time difference and deduplicate(keep the closest record for each ecg_time)
ca_unique = ca_df.sort_values('time_diff').groupby(['subject_id', 'ecg_time']).head(1)
k_unique = k_df.sort_values('time_diff').groupby(['subject_id', 'ecg_time']).head(1)

# Combine K and Ca (allow independence)
final_df = pd.merge(
    ca_unique[['subject_id', 'ecg_time', 'value', 'time_diff', 'study_id', 'hea_path']],
    k_unique[['subject_id', 'ecg_time', 'value', 'time_diff', 'study_id', 'hea_path']],
    on=['subject_id', 'ecg_time', 'study_id', 'hea_path', 'time_diff'],
    how='outer',
    suffixes=('_ca', '_k')
)

# Rename columns
final_df.rename(columns={
    'value_ca': 'calcium',
    'value_k': 'potassium'
}, inplace=True)

# Save
output_path = r'C:\Users\DELL\Desktop\DataECG\matched_lab_ecg_unique_demo.csv'
final_df.to_csv(output_path, index=False)

both_df = final_df.dropna(subset=['calcium', 'potassium'])
output_path_both = r'C:\Users\DELL\Desktop\DataECG\matched_lab_ecg_both_demo.csv'
both_df.to_csv(output_path_both, index=False)

# Output
print(f"Matching completed, results have been saved to: {output_path, both_df}")
print("\nFinal statistics:")
print(f"- Unique ECG records: {len(final_df)}")
print(f"- Records with calcium: {final_df['calcium'].notna().sum()}")
print(f"- Records with potassium: {final_df['potassium'].notna().sum()}")
print(f"- Records with both calcium and potassium: {final_df[['calcium', 'potassium']].notna().all(axis=1).sum()}")


Matching completed, results have been saved to: ('C:\\Users\\DELL\\Desktop\\DataECG\\matched_lab_ecg_unique_demo.csv',       subject_id            ecg_time calcium       time_diff  study_id  \
1       10000032 2180-07-23 09:54:00     9.3 0 days 11:51:00  44458630   
2       10000032 2180-08-06 09:07:00     8.6 0 days 02:31:00  49036311   
3       10000117 2183-09-18 13:52:00     9.9 0 days 01:26:00  48446569   
8       10000764 2132-10-16 09:54:00     8.1 0 days 03:51:00  40539087   
10      10000826 2146-12-12 08:42:00     8.2 0 days 03:32:00  40695233   
...          ...                 ...     ...             ...       ...   
3861    10070626 2157-06-03 02:01:00       8 0 days 01:50:00  41184490   
3862    10070626 2157-06-03 14:58:00     8.3 0 days 14:54:00  44445866   
3864    10070701 2156-10-02 16:10:00       9 0 days 00:00:00  48420484   
3865    10070701 2156-10-03 14:51:00     8.3 0 days 03:31:00  49084196   
3867    10070735 2128-02-26 14:21:00     8.8 0 days 00:54:00  47870

In [15]:
# Seperate Ca and K（Keep charttime）
ca_df = merged[merged['itemid'] == 50893][['subject_id', 'charttime', 'ecg_time', 'value', 'time_diff', 'study_id', 'hea_path']].copy()
k_df = merged[merged['itemid'] == 50971][['subject_id', 'charttime', 'ecg_time', 'value', 'time_diff', 'study_id', 'hea_path']].copy()

# Sort by Time difference, duplicate
ca_unique = ca_df.sort_values('time_diff').groupby(['subject_id', 'ecg_time']).head(1)
k_unique = k_df.sort_values('time_diff').groupby(['subject_id', 'ecg_time']).head(1)

final_df = pd.merge(
    ca_unique.rename(columns={'charttime': 'charttime_ca', 'value': 'calcium'}),
    k_unique.rename(columns={'charttime': 'charttime_k', 'value': 'potassium'}),
    on=['subject_id', 'ecg_time', 'study_id', 'hea_path', 'time_diff'],
    how='outer'
)

final_df['ecg_time'] = pd.to_datetime(final_df['ecg_time'])
final_df['time_diff'] = final_df['time_diff'].astype('timedelta64[ns]')

# Save
output_path = r'C:\Users\DELL\Desktop\DataECG\matched_lab_ecg_with_charttime_demo.csv'
final_df.to_csv(output_path, index=False)



In [16]:
# Verify if Ca and K were tested at the same charttime
# Add two columns：is_same_time 和 time_diff_between_ca_k
final_df['is_same_time'] = (final_df['charttime_ca'] == final_df['charttime_k']).astype('boolean')
final_df['time_diff_between_ca_k'] = (final_df['charttime_ca'] - final_df['charttime_k']).abs()

# Turn into hours
final_df['time_diff_between_ca_k'] = final_df['time_diff_between_ca_k'].dt.total_seconds() / 3600 

print("\nTest statistics:")
print(f"- Number of records where Ca and K were tested at the same time: {final_df['is_same_time'].sum()}")
print(f"- Maximum time difference between Ca and K tests: {final_df['time_diff_between_ca_k'].max():.2f} hours")


Test statistics:
- Number of records where Ca and K were tested at the same time: 1929
- Maximum time difference between Ca and K tests: 0.00 hours
